### 9.1二手房数据处理

#### 1.数据导入与观测

In [2]:
import pandas as pd
import numpy as np
df=pd.read_excel('pandas_二手房.xlsx',index_col=0) #导入数据，索引为第0列
df.info()  #查看dataframe字段格式

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2939 entries, 107002321157 to 107000059163
Data columns (total 16 columns):
总价      2939 non-null float64
小区      2939 non-null object
区域      2939 non-null object
房屋户型    2939 non-null object
所在楼层    2939 non-null object
建筑面积    2939 non-null object
户型结构    2939 non-null object
建筑类型    2939 non-null object
房屋朝向    2939 non-null object
建筑结构    2939 non-null object
装修情况    2939 non-null object
梯户比例    2939 non-null object
交易权属    2939 non-null object
产权所属    2939 non-null object
挂牌时间    2939 non-null object
上次交易    2939 non-null object
dtypes: float64(1), object(15)
memory usage: 390.3+ KB


In [13]:
df.head()  #查看前5行数据
df.head(10) #查看前10行数据
df.tail()  #查看后5行数据
df.describe() #查看描述性统计结果

#### 2.缺失值处理

In [4]:
#方法一 利用applymap方法遍历处理缺失值
def replace_with_nan(s):
    if s=='':
        return np.nan
    else:
        return s
df=df.applymap(replace_with_nan)

In [6]:
#方法二 在dataframe读入时指定需要替换为Nan的值
df=pd.read_excel('pandas_二手房.xlsx',index_col=0,na_values='暂无数据')
df.isnull().sum()

总价         0
小区         0
区域         0
房屋户型       0
所在楼层       0
建筑面积       0
户型结构    2939
建筑类型      21
房屋朝向     154
建筑结构       0
装修情况       0
梯户比例       0
交易权属       0
产权所属    2723
挂牌时间       0
上次交易      65
dtype: int64

#### 3.数据转换

In [8]:
#将字符串日期转换为datetime格式的日期
df['挂牌时间']=pd.to_datetime(df['挂牌时间'])
df['上次交易']=pd.to_datetime(df['上次交易'])

In [9]:
#通过字符串对象的split函数，对字符串进行分割，并建立新的字段
df['建筑面积']=df['建筑面积'].map(lambda s :s.split('㎡')[0])
df['区']=df['区域'].map(lambda s :s.split(' ')[0])  #取区
df['板块']=df['区域'].map(lambda s :s.split(' ')[1])  #取板块
df['环线']=df['区域'].map(lambda s :s.split(' ')[2])  #取环线
df['室']=df['房屋户型'].map(lambda s :int(s.split('室')[0]))   #取几室
df['厅']=df['房屋户型'].map(lambda s :int(s.split('厅')[0].split('室')[1])) #取几厅
df['厨']=df['房屋户型'].map(lambda s :int(s.split('厨')[0].split('厅')[1])) #取几厨
df['卫']=df['房屋户型'].map(lambda s :int(s.split('厨')[1].split('卫')[0])) #取几卫
df['楼层位置']=df['所在楼层'].map(lambda s :s.split('楼')[0])   #取楼层
df['楼层总数']=df['所在楼层'].map(lambda s :int(s.split('共')[1].split('层')[0]))   #取楼层数
df['电梯数']=df['梯户比例'].map(lambda s :s.split('梯')[0])  #取几梯
df['楼层户数']=df['梯户比例'].map(lambda s :s.split('梯')[1].split('户')[0]) #取几户
del df['区域']
del df['房屋户型']
del df['所在楼层']
del df['梯户比例']

In [10]:
df['电梯数'].unique()

array(['一', '两', '三', '四', '六', '五'], dtype=object)

In [4]:
#将中文数字转换为阿拉伯数字
def cta(s):
    con_table ={'零':0, '一':1, '二':2, '两':2, '三':3, '四':4, '五':5, '六':6, '七':7, '八':8, '九':9, '十':10}
    n=len(s)
    if n==3:
        return con_table[s[0]]*10+con_table[s[2]]
    if n==2:
        return con_table[s[0]]+con_table[s[1]]
    if n==1:
        return con_table[s[0]]
df['电梯数']=df['电梯数'].map(cta)
df['楼层户数']=df['楼层户数'].map(cta)

#### 4.数据重塑

In [12]:
df['交易权属'].unique()

array(['商品房', '动迁安置房', '售后公房'], dtype=object)

In [14]:
#将‘交易权属’字段转换为虚拟变量
pd.get_dummies(df['交易权属']).head()

,动迁安置房,售后公房,商品房
房屋编号,,,
107002321157,0.0,0.0,1.0
107000752483,0.0,0.0,1.0
107000488257,0.0,0.0,1.0
107000285319,0.0,0.0,1.0
107001668181,1.0,0.0,0.0


In [15]:
#将包含虚拟变量的dataframe与原dataframe在行索上对齐连接
df=df.join(pd.get_dummies(df['交易权属']))
del df['交易权属']

In [16]:
#数据透视表
pd.pivot_table(df,values=['总价'],index='区',columns='环线',aggfunc='mean').head()

总价                                                
环线                   中环至外环         内环内       内环至中环         外环外
区                                                             
嘉定         NaN  421.423077         NaN         NaN  335.964286
奉贤  285.363636         NaN         NaN         NaN  239.017391
宝山  570.209302  413.711934         NaN  458.682927  369.762500
徐汇         NaN  479.843750  992.338462  560.015748  512.500000
普陀         NaN  512.085714  872.561798  545.836538  275.545455

### 9.2 职位数据处理

#### 1.数据导入与观测

In [17]:
import pandas as pd
import numpy as np
df=pd.read_excel('pandas_职位招聘.xls',index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4980 entries, 4268845 to 3749022
Data columns (total 12 columns):
positionName       4980 non-null object
city               4980 non-null object
createTime         4980 non-null object
companyFullName    4980 non-null object
education          4980 non-null object
industryField      4980 non-null object
companySize        4980 non-null object
firstType          4980 non-null object
secondType         4980 non-null object
salary             4980 non-null object
workYear           4980 non-null object
description        4962 non-null object
dtypes: object(12)
memory usage: 505.8+ KB


#### 2.缺失值处理

In [18]:
df=df.dropna()
df.isnull().sum()

positionName       0
city               0
createTime         0
companyFullName    0
education          0
industryField      0
companySize        0
firstType          0
secondType         0
salary             0
workYear           0
description        0
dtype: int64

#### 3.数据转换

In [19]:
#剔除'description'列后显示dataframe
df.drop('description',axis=1).head()

,positionName,city,createTime,companyFullName,education,industryField,companySize,firstType,secondType,salary,workYear
positionId,,,,,,,,,,,
4268845,22989-大数据和人工智能高级开发工程师,深圳,2018-03-15 15:27:44,腾讯科技(深圳)有限公司,本科,"移动互联网 ,游戏",2000人以上,开发/测试/运维类,后端开发,25k-50k,3-5年
3519814,大数据--资深开发工程师,北京,2018-03-16 10:14:41,快看世界（北京）科技有限公司,本科,"移动互联网,文化娱乐",150-500人,开发/测试/运维类,数据开发,30k-50k,3-5年
4190526,大数据/AI高级产品经理,北京,2018-03-17 20:51:57,北京车之家信息技术有限公司,本科,"移动互联网,文化娱乐",2000人以上,产品/需求/项目类,产品设计/需求分析,25k-35k,3-5年
3996747,大数据与机器学习研发专家,上海,2018-03-17 18:11:01,北京三快在线科技有限公司,本科,"移动互联网,O2O",2000人以上,开发/测试/运维类,数据开发,20k-40k,3-5年
4180225,数据与智能引擎总监,深圳,2018-03-16 10:36:35,深圳平安讯科技术有限公司,本科,移动互联网,150-500人,开发/测试/运维类,数据开发,30k-50k,5-10年


In [20]:
#使用apply函数按分隔符分割列
df['industryField']=df['industryField'].apply(lambda e: e.split(',')) # 所在行业转换成列表
df['firstType']=df['firstType'].apply(lambda e: e.split('/'))  #职位大类转换成列表
df['secondType']=df['secondType'].apply(lambda e: e.split('/')) #职位小类转换成列表

In [21]:
#查看各列非重复值个数
print(len(df['workYear'].unique()))
print(len(df['companySize'].unique()))
print(len(df['education'].unique()))
print(len(df['salary'].unique()))

6
6
5
230


In [22]:
#salary字段的单位有k何K两种，分割salary字段
import re
df['salary']=df['salary'].apply(lambda e: int(re.split('[k,K]', e)[0]))

In [23]:
#日期时间字段转换
df['createTime']=pd.to_datetime(df['createTime'])

### 9.3 职位描述中文自然语言处理

#### 1. 中文分词词库jieba简介

In [25]:
#jieba的三种分词模式
import jieba
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("全模式: " + "/ ".join(seg_list))  
seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("精确模式: " + "/ ".join(seg_list))  
seg_list = jieba.cut("他来到了网易杭研大厦")  
print("默认为精确模式: "+", ".join(seg_list))
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所") 
print("搜索引擎模式: " + ", ".join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lzp\AppData\Local\Temp\jieba.cache
Loading model cost 0.765 seconds.
Prefix dict has been built succesfully.


全模式: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
精确模式: 我/ 来到/ 北京/ 清华大学
默认为精确模式: 他, 来到, 了, 网易, 杭研, 大厦
搜索引擎模式: 小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所


In [26]:
#返回相应分词的词性
import jieba.posseg
seg_list = jieba.posseg.cut("小明硕士毕业于中国科学院计算所")
for word, flag in seg_list:
    print(str(word)+ ' '+ str(flag))

小明 nr
硕士 n
毕业 n
于 p
中国科学院 nt
计算所 n


#### 2. 对每个职位description分词

In [27]:
#首先定义分词词性过滤函数filter_flag
def filter_flag(seg):
    include=['n','v','eng']
    wordf=[]
    for i in seg:
        if i.flag in include:
            wordf.append(i.word)
    return wordf

In [28]:
#第二步 遍历df['description']序列，对每个职位的描述文本进行分词
import jieba.posseg
wordlist=[]
for rec in df['description']:
    word=filter_flag(jieba.posseg.cut(rec))
    wordlist.append(' '.join(word))
wordlist[:5] #显示分词后列表wordlist的前5个元素

['职位 描述 岗位职责 负责 数据 AI 商业化 产品 交付 了解 客户 需求 保质 完成 计划 协助 解决方案 团队 完成 客户 化 方案 实施 协助 运维 团队 保障 交付 产品 岗位 要求 Linux C PHP Python Java 编程语言 技术开发 相关 经验 熟悉 质量 流程 有 云 计算 产品 SaaS 交付 经验 能 独立 完成 模块 开发 交付 有 并发 大容量 后台 经验 熟悉 计算 大 数据 相关 技术 数据 机器 学习 图片 识别 语音 识别 相关 背景 经验 有 应用 开发 经验 例如 Container Kubernetes DevOps CD CI 具备 沟通 表 能力 团队 精神 有 责任心 执行 能力 计算机相关 专业',
 '职位 描述 职责 深入 理解 业务 需求 业务 建模 系统 架构 满足 业务 需求 维护 大 数据 集群 架构 业务 需求 规划 集群 架构 扩展 解决 遇到 技术难题 指导 同事 开发 任职 条件 计算机相关 专业本科 编程 基础 有 数据 开发 经验 Hadoop 生态 体系 架构 有 深入 理解 熟悉 Spark Hive 开源 工具 Flume 收集 体系 用户 行为 采集 海量 数据处理 数据 建模 业务 理解 方面 有 经验 擅长 挖掘 梳理 需求 具有 逻辑性 思维 能 协调 需求 能 学习 开源 框架 知识 体系 应用 到 架构 体系 能 学习业务 领域 知识 具有 产品 思维 职业 素养 看 欢迎',
 '职位 描述 岗位职责 熟悉 汽车 大 数据 平台 AI 相关 技术 能力 产品 附能 业务 数据 部门 业务 方向 规划 产品 解决方案 团队 迭代 BU 业务部门 公司 沟通 以求 赢 任职 要求 熟悉 数据 平台 AI 产品 技术 原理 互联网 公司 数据 AI 相关 经验 擅长 沟通 意识',
 '职位 描述 岗位职责 负责 数据安全 平台 平台 数据处理 数据挖掘 岗位 要求 Java 语言 擅长 使用 开源 框架 比如 elk kafka storm spark 数据处理 框架 掌握 实现 原理 擅长 使用 机器 学习 框架 TensorFlow 掌握 原理 熟悉 机器 学习 算法 神经网络 SVM 熟悉 回归 分析模型 规则 挖掘 分类 聚类 算法 协同 过滤 算法 数据

In [29]:
#第三步，将分词列表转换为词频矩阵
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer()
word_array=vec.fit_transform(wordlist)
print('列：'+str(len(word_array.toarray()[0]))+' 行：'+str(len(word_array.toarray())))

列：9152 行：4962


In [30]:
#生成前3个职位的词频矩阵，并返回词频矩阵列标签
vec=CountVectorizer()
word_array=vec.fit_transform(wordlist[:3])
print('列：'+str(len(word_array.toarray()[0]))+' 行：'+str(len(word_array.toarray())))
print(vec.get_feature_names())
print(word_array.toarray())

列：129 行：3
['ai', 'bu', 'cd', 'ci', 'container', 'devops', 'flume', 'hadoop', 'hive', 'java', 'kubernetes', 'linux', 'php', 'python', 'saas', 'spark', '专业', '专业本科', '业务', '业务部门', '了解', '互联网', '交付', '产品', '以求', '任职', '体系', '例如', '保质', '保障', '公司', '具备', '具有', '协助', '协调', '原理', '同事', '后台', '商业化', '团队', '图片', '基础', '大容量', '学习', '学习业务', '完成', '实施', '客户', '岗位', '岗位职责', '工具', '平台', '并发', '应用', '建模', '开发', '开源', '思维', '意识', '执行', '扩展', '技术', '技术开发', '技术难题', '指导', '挖掘', '描述', '擅长', '收集', '数据', '数据处理', '方向', '方案', '方面', '机器', '条件', '架构', '框架', '梳理', '模块', '欢迎', '汽车', '沟通', '流程', '海量', '深入', '满足', '熟悉', '独立', '理解', '生态', '用户', '相关', '知识', '精神', '系统', '素养', '经验', '维护', '编程', '编程语言', '职业', '职位', '职责', '背景', '能力', '行为', '要求', '规划', '解决', '解决方案', '计划', '计算', '计算机相关', '识别', '语音', '负责', '责任心', '质量', '运维', '迭代', '逻辑性', '遇到', '部门', '采集', '附能', '集群', '需求', '领域']
[[1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0 4 3 0 0 0 1 1 1 0 1 0 2 0 0 0
  1 1 3 1 0 1 1 0 3 1 2 1 1 0 0 1 1 0 2 0 0 0 1 0 1 1 0 0 0 1 0 0 3 0 